# Getting Started

## Install the SingleStore package

The SingleStore package can be installed the following way:
```
pip install singlestore
```

It can also be installed using `conda` for Anaconda users:
```
conda install -c singlestore singlestore
```

In addition, you can install the SQLAlchemy plugin with the following:
```
pip install singlestore[sqlalchemy]
```

## Import SingleStore

In [1]:
import singlestore as s2

## Create a connection

This function is compliant with the Python DB-API 2.0. In addition, it allows you to use a URL
as a connection string rather than individual parameters. Parameters may also be set using
environment variable (e.g., `SINGLESTORE_HOST`, `SINGLESTORE_USER`, `SINGLESTORE_PASSWORD`, etc.).

In [2]:
conn_url = 'root:@127.0.0.1/x_db'

In [3]:
conn = s2.connect(conn_url)

The URL in the code above can take various forms. One feature of it is that it can be used to specify alternate MySQL drivers or the HTTP API (shown below).
```
http://root:@localhost:9000/x_db
```

The default connector is `mysql.connector`.
```
mysqlconnector://root:@localhost/x_db
```

Other drivers can be specified as well, including `mysqldb`, `pymysql`, `mariadb`, and `cymysql`. These will be unsupported modes however. They are 
simply built in as part of the architecture in case we want to change to a different one later on. All of the connection parameters and behaviors
are normalized so that they are interchangeable.
```
mysqldb://root:@localhost/x_db
pymysql://root:@localhost/x_db

```

### Environment Variables

Connection URLs can also be set using an environment variable. This allows you to create environments that already have connection parameters
embedded in them so that new connections can be made without specifying any parameters.

In [4]:
import os

os.environ['SINGLESTORE_URL'] = conn_url

### Cursors

Most interaction with databases are done using cursors. To create a cursor from a connection, you use the `cursor` method.

In [5]:
cur = conn.cursor()

## Create tables and insert data

Using the cursor, we can execute commands for creating tables and inserting data.

### Reset Tables

In [6]:
cur.execute(r'drop table if exists departments')
cur.execute(r'drop table if exists employees')
cur.execute(r'drop table if exists salaries')

cur.execute(r'''
create table if not exists departments (
  id int,
  name varchar(255),
  primary key (id)
);''')

cur.execute(r'''
create table if not exists employees (
  id int,
  deptId int,
  managerId int,
  name varchar(255),
  hireDate date,
  state char(2),
  primary key (id)
);''')

cur.execute(r'''
create table if not exists salaries (
  employeeId int,
  salary int,
  primary key (employeeId)
);''')

### Insert Data Using Named Parameters

Using named parameters such as `:foo` is the default parameter format in queries. This requires dictionaries for the parameter structure.

In [7]:
cur.executemany(r'insert into departments(id, name) values (:id, :name)',
    [
        dict(id=1, name='Marketing'),
        dict(id=2, name='Finance'),
        dict(id=3, name='Sales'),
        dict(id=4, name='Customer Service'),
    ]
)

### Insert Data Using Positional Parameters

You can also use positional parameters which specify the one-based index in a list or tuple.

In [8]:
cur.executemany(r'insert into employees (id, deptId, managerId, name, hireDate, state) '
                r'values (:1, :2, :3, :4, :5, :6)',
    [
        (1, 2, None, "Karly Steele", "2011-08-25", "NY"),
        (2, 1, 1, "Rhona Nichols", "2008-09-11", "TX"),
        (3, 4, 2, "Hedda Kent", "2005-10-27", "TX"),
        (4, 2, 1, "Orli Strong", "2001-07-01", "NY"),
        (5, 1, 1, "Leonard Haynes", "2011-05-30", "MS"),
        (6, 1, 5, "Colette Payne", "2002-10-22", "MS"),
        (7, 3, 4, "Cooper Hatfield", "2010-08-19", "NY"),
        (8, 2, 4, "Timothy Battle", "2001-01-21", "NY"),
        (9, 3, 1, "Doris Munoz", "2008-10-22", "NY"),
        (10, 4, 2, "Alea Wiggins", "2007-08-21", "TX"),
    ]
)

### Insert Data From DataFrame

In addition to this method, you can use the `pandas.DataFrame.to_sql` method with a SQLAlchemy SingleStore connection object.

In [9]:
import pandas as pd

df = pd.DataFrame( 
    [
        (1, 885219), 
        (2, 451519), 
        (3, 288905), 
        (4, 904312), 
        (5, 919124),
        (6, 101538), 
        (7, 355077), 
        (8, 900436), 
        (9, 41557), 
        (10, 556263),
    ],
    columns=['employeeId', 'salary'])

cur.executemany(r'insert into salaries (employeeId, salary) '
                r'values (:1, :2)', df)

conn.commit()

# Querying data

Here we are executing a simple `SELECT` operation.

In [ ]:
cur.execute('select name from employees')

There are various ways of fetching data including the `fetchone()`, `fetchmany()`, and `fetchall()` methods
description in the DB-API. In addition, you can simply iterate over the cursor itself.

In [ ]:
for item in cur:
    print(item)

By default, results come back in tuples, but there are other result structure options as well, they include `namedtuple`, `dict`, and `dataframe`.

In [ ]:
conn = s2.connect(results_format='dataframe')

In [ ]:
cur = conn.cursor()
cur.execute('select name from employees')
cur.fetchall()

The following is a query using parameter substitution. While the underlying connection object is a `mysql.connector`-based
object which takes the pyformat parameter format, we are using the `sqlparams` package to allow us to 
use whatever substitution form we want. Since the HTTP API uses question marks for parameters, we have
adopted that format here as well.

In [ ]:
cur.execute('select name, hireDate from employees where name like :1', ['%Rhona%'])

The `description` field returns information about the query results. It includes fields such as `name`, `type_code`,
`display_size`, `internal_size`, etc. These are defined in the DB-API, but are not all populated.

In [ ]:
cur.description

In [ ]:
cur.fetchall()

## Server Variables

Server variables can be accessed through a number of dictionary-like members on the connection. The members are `globals`, `locals`, `cluster_globals`, `cluster_locals` when a scope is specified, or `vars` and `cluster_vars` for all variables regardless of scope.

In [ ]:
dict(conn.globals)

In [ ]:
conn.globals.enable_external_functions = True

In [ ]:
conn.globals.enable_external_functions

In [ ]:
cur.execute('show variables like "enable_external_functions"')
cur.fetchall()

## Enabling the HTTP API

The HTTP API can be enabled using the `conn.enable_http_api` method. This method can optionally set the port number.
If a port number is not specified, the existing setting will be used. The port number is returned.

In [10]:
conn.enable_http_api(port=8100)

8100

## Create an HTTP connection

In [11]:
s2.options.results.format = 'dataframe'

In [12]:
http_conn = s2.connect('http://root:@localhost:8100/x_db')

In [13]:
http_cur = http_conn.cursor()

## Query data using the HTTP connection

In [14]:
http_cur.execute('select name from employees')

In [15]:
http_cur.description

[Description(name='name', type_code=253, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0)]

In [16]:
http_cur.fetchall()

,name
0,Karly Steele
1,Rhona Nichols
2,Hedda Kent
3,Orli Strong
4,Leonard Haynes
5,Colette Payne
6,Cooper Hatfield
7,Timothy Battle
8,Doris Munoz
9,Alea Wiggins


In [17]:
http_cur.execute('select name, hireDate from employees where name like :1', ['%Rhona%'])

In [18]:
http_cur.description

[Description(name='name', type_code=253, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0),
 Description(name='hireDate', type_code=10, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True, flags=0, charset=0)]

In [19]:
df = http_cur.fetchall()
df

,name,hireDate
0,Rhona Nichols,2008-09-11


## Cluster Management

The objects allow you to manage clusters and create database connections to those clusters. In order for this call
to work, you either need to pass in a cluster management API token or have one set in your `SINGLESTORE_CLUSTER_MANGER_TOKEN` environment variable.

In [20]:
cm = s2.manage_cluster()

### Get Current Clusters and Regions

In [21]:
cm.clusters

[]

In [22]:
cm.regions

[Region(id='09754160-66cd-4e8e-b223-1d9e1507b4ba', provider='Azure', region='East US (Virginia)'),
 Region(id='1c1de314-2cc0-4c74-bd54-5047ff90842e', provider='GCP', region='US West 1 (Oregon)'),
 Region(id='22a913e1-ace5-4362-afa0-3d21a01d6d1c', provider='GCP', region='US Central 1 (Iowa)'),
 Region(id='3186a27b-87d5-4b84-a93b-90c57fd2be74', provider='GCP', region='Asia South 1 (Mumbai)'),
 Region(id='35551562-6d39-4e14-8747-0f42114a330b', provider='AWS', region='Europe West 1 (Ireland)'),
 Region(id='3de9e276-1c93-4987-9463-5700aa08a837', provider='AWS', region='US West 2 (Oregon)'),
 Region(id='57ba8143-d4cc-470a-901f-871c684ee2bc', provider='AWS', region='Europe North 1 (Stockholm)'),
 Region(id='6e218f48-a71f-4521-8c7c-53d0ef087a42', provider='GCP', region='Europe West 1 (Belgium)'),
 Region(id='7b9163df-67b0-45fb-985b-e571fbcb3792', provider='AWS', region='South America East 1 (Sao Paulo)'),
 Region(id='7e7ffd27-20f7-44b6-87e6-e72828a81ac7', provider='AWS', region='Europe Central

### Create a Cluster

In [ ]:
import secrets
password = secrets.token_urlsafe(20)

In [ ]:
clus = cm.create_cluster('Demo Cluster', 
                         region_id=cm.regions[0],
                         admin_password=password, 
                         firewall_ranges=['0.0.0.0/0'],
                         expires_at='1h', wait_on_active=True)
clus

### Make a Database Connection to Cluster

In [ ]:
with clus.connect(user='admin', password=password) as conn:
    with conn.cursor() as cur:
        cur.execute('show databases')
        print(cur.fetchall())

In [ ]:
clus.terminate()